# Introduction 

Before I settled in California, I was always thinking to adopt a cat. So after I finally settle, I check petfinder, if you're not familiar with it, it's a platform that helps you to find pets that are available for adoption.

I live in westwood and there are plenty of shelters but I end up adopt from a shelter 60 miles away after I saw the picture of our cat in Petfinder. Without any reponse from the shelter, just one single picture in petfinder, we drove there immediately after we saw his cute cat paws. Before that we had been to many shelters and met so many stray animals, all after we saw some cute animal friends we would want to meet. Petfinder.com has options for shelters and rescuers to set up the pet profile, their basic features, but from my experience, the first sight from the pictures matters the most. I think this project can help improve people find their ideal animal friends, as well as more animal waiting to be adopted find their forever home. It can also guide shelters and rescuers to take better pictures to increase the chance that these animals being favored at the first sight. 
I also adopted a puppy in my home country China, where animal rescuing growing rapidly these few years. But we are not doing enough. I was also volunteering and helping local shelters, and writing a lot to government facilities for animal protection laws. I really hope this project can provide some insights, at least for myself, to improve animal rescue and promote stray animal adoption.

# Method

Here's the link of data sources. [Final Project Datasets: Petfinder Popularity of shelter pet photos prediction](https://www.kaggle.com/c/petfinder-pawpularity-score/data "Final Project Datasets: Petfinder Popularity of shelter pet photos prediction")
### - The Pawpularity Score 
The Pawpularity Score is derived from each pet profile's page view statistics at the listing pages, using an algorithm that normalizes the traffic data across different pages, platforms (web & mobile) and various metrics. Duplicate clicks, crawler bot accesses and sponsored profiles are excluded from the analysis.
### - Test and Train
The data is consisted of test data and train data. To be noted, the train.csv and test.csv files contain metadata for photos in the training set and test set, respectively. They featured each photo features such as photo quality, pet action, etc.

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from glob import glob

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
train_jpg = glob("train/*.jpg")
test_jpg = glob("test/*.jpg")

## About the data

In [ ]:
train_df.shape

In [ ]:
train_df.info()

In [ ]:
test_df.shape

In [ ]:
type(train_jpg)

In [ ]:
len(train_jpg)

In [ ]:
train_jpg[0:5]

# About the train data

In [ ]:
train_df.head()
# the 0/1 shows no/yes of each feature

In [ ]:
train_df.describe()

In [ ]:
train_df[['Pawpularity']].describe()

#### distribution

In [ ]:
sns.histplot(data=train_df, x='Pawpularity', bins=100)
plt.axvline(train_df['Pawpularity'].mean(), c='red', ls='-',label='Mean Pawpularity')
plt.axvline(train_df['Pawpularity'].median(),c='green',ls='-',label='Median Pawpularity')
plt.legend()
plt.show()
# little peak at the beginning and almost 300 100score. There's trend in the data

#### I wanted to plot this graph with each feature and see how they relate to the overall data. Found this code below and I think it's so efficient! 

In [ ]:
feature_variables = train_df.columns.values.tolist()

#for each of the feature variables, doesn't include Id and Pawpularity by using [1:-1]
#show a boxplot and distribution plot against pawpularity

for variable in feature_variables[1:-1]:
    fig, ax = plt.subplots(1,2)
    sns.boxplot(data=train_df, x=variable, y='Pawpularity', ax=ax[0])
    sns.histplot(train_df, x="Pawpularity", hue=variable, kde=True, ax=ax[1])
    plt.suptitle(variable, fontsize=20, fontweight='bold')

# Image data

In [ ]:
print(train_jpg[0])

In [ ]:
image_path = train_jpg[0]
image_array = plt.imread(image_path) 
print(image_array.shape)

In [ ]:
plt.imshow(image_array)

In [ ]:
for x in range(5):
    image_path = train_jpg[x]
    image_array = plt.imread(image_path) 
    plt.imshow(image_array)
    plt.title(x) 
    plt.show()

#### To show the pawpularity score of each picture

In [ ]:
from pathlib import Path

In [ ]:
for x in range(5):
    image_path = train_jpg[x]
    
    # to get the matching score
    id_stem = Path(image_path).stem
    id_stem_series = train_df.loc[train_df['Id'] == id_stem,'Pawpularity']
    pawpularity_match_id = id_stem_series.iloc[0]
    
    image_array = plt.imread(image_path) 
    plt.imshow(image_array)
    title = image_path + ' the score is ' +str(pawpularity_match_id)
    plt.title(title) 
    plt.show()

# Models

In my Machine Learning with R class I learned several models that are suitable for this case. Last week I just learned decision trees and found similar cases so I'll try use some models from that. There are many models in decision trees that I can use. I already replicate the course lab in R so I wish it can work out in Python.

Another reason that I choose decision tree is that in the EDA part, we can see that the data is not really "identicle". Pictures that actually looks alike could have very different score. I'm curious about why. Tree-based methodstypically are not competitive with the best supervised learning approaches but very useful for interpretation. So I think this will still work in this case.

This course also already covered ML and DL, in this project I started with simple ones: regression trees and classification trees, but the result didn't seem well, so I also did boost and I choose the natural gradient boosting.

Also this competetion only has five test data so it wouldn't look as good in the graphs. I'll just split train and test data test_size=0.2 myself.

## Regression Trees

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier

In [ ]:
y = train_df['Pawpularity']

# exclude Id and Pawpualarirty for the feature variables
X = train_df.drop(['Id','Pawpularity'],axis=1)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=5)

In [ ]:
tree_reg = DecisionTreeRegressor(max_depth = 3, min_samples_split = 10)
tree_reg.fit(x_train, y_train)
tree_reg_pred = tree_reg.predict(x_test)

In [ ]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_test, tree_reg_pred))

Not very good result

#### Visualize the tree

In [ ]:
fig = plt.figure(figsize=(15,5))
plot = tree.plot_tree(tree_reg, feature_names=x_train.columns.values.tolist(), filled=True)

#### test the model

In [ ]:
import matplotlib.patches as mpatches

In [ ]:
def testthemodel(y_test,y_pred,title):
    if max(y_test) >= max(y_pred):
        model_range = int(max(y_test))
    else:
        model_range = int(max(y_pred))
    plt.scatter(range(len(y_test)), y_test, color='green')
    plt.scatter(range(len(y_pred)), y_pred, color='red')
    plt.ylabel('Pawpularity ')
    plt.legend(handles = [mpatches.Patch(color='red', label='prediction'),mpatches.Patch(color='g', label='test')])
    plt.show()
    return

In [ ]:
testthemodel(y_test, tree_reg_pred, "Test vs Predicted")
plt.figure(figsize=(12,4))
sns.histplot(tree_reg_pred,color='r',stat='probability', kde=True)
sns.histplot(y_test,color='g',alpha=0.3,stat='probability', kde=True)
plt.legend(labels=['prediction','Test'])
plt.title('Test vs Predicted')
plt.show()

## Classification Trees

In [ ]:
tree_clf = DecisionTreeClassifier(max_depth = 4, min_samples_split = 10)
tree_clf.fit(x_train, y_train)
tree_clf_pred = tree_clf.predict(x_test)

In [ ]:
np.sqrt(mean_squared_error(y_test, tree_clf_pred))

Still look bad. Should keep trying other model for the final submission.

In [ ]:
testthemodel(y_test, tree_clf_pred, "Test vs Predicted")

plt.figure(figsize=(12,4))
sns.histplot(tree_clf_pred,color='r',alpha=0.3,stat='probability', kde=True)
sns.histplot(y_test,color='g',alpha=0.3,stat='probability', kde=True)
plt.legend(labels=['prediction','test'])
plt.title('Test vs Predict Distribution')
plt.show()

Actually even if the sqrt is bigger for classification, the graph somehow looks a bit more accurate than regression.

## Natural Gradient Boosting

In [11]:
from ngboost import NGBRegressor

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_test, Y_train, Y_test = train_test_split(train[feature_columns], Y, test_size=0.2)

ngb = NGBRegressor().fit(X_train, Y_train)
Y_preds = ngb.predict(X_test)
Y_dists = ngb.pred_dist(X_test)


mean_squared_error(Y_preds, Y_test)

NameError: name 'train_test_split' is not defined

# Image models

I saw most people were using pytorch but it looks much more complicated. So I'll stay with the course and use tensorflow.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.client import device_lib

In [ ]:
def train_id_to_path(x):
    return 'train/' + x + ".jpg"
def test_id_to_path(x):
    return 'test/' + x + ".jpg"

In [ ]:
train_df.head()

In [ ]:
train = pd.read_csv('train.csv')
train = train.drop(['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'],axis=1)

In [ ]:
test = pd.read_csv('test.csv')
test = test.drop(['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'],axis=1)

In [ ]:
train["img_path"] = train["Id"].apply(train_id_to_path)
test["img_path"] = test["Id"].apply(test_id_to_path)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
image_height = 100
image_width = 100

In [ ]:
def path_to_tensor(image_path):
    raw = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(raw, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, (image_height, image_width))
    return image

In [ ]:
print(train['img_path'][0])

In [ ]:
og_example_image = plt.imread('train/0007de18844b0dbbb5e1f607da0606e0.jpg') 

In [ ]:
print(og_example_image.shape)
plt.imshow(og_example_image)
plt.show()

In [ ]:
X = []
for img in train['img_path']:
    new_img_tensor = path_to_tensor(img)
    X.append(new_img_tensor)
    
print(type(X),len(X))
X = np.array(X)
print(type(X),X.shape)